In [ ]:
# Montar Google Drive para acceder a los archivos
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
import torch
import os
import shutil  # Importar para mover archivos
import matplotlib.pyplot as plt

# Configuración del dispositivo (GPU o CPU) usando PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo en uso: {device}")

# Función para calcular el brillo promedio de una imagen
def calculate_brightness(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.mean(gray_image)

# Función para calcular el contraste de una imagen
def calculate_contrast(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.std(gray_image)

# Directorio de imágenes en Google Drive
dataset_dir = '/content/drive/My Drive/LauCows/VacasLauPixel'  # Ajusta esto a tu carpeta en Google Drive
output_dir = '/content/drive/My Drive/LauCows/VacasLauNoOutliers'  # Nueva carpeta para no outliers en Google Drive

# Crear carpeta de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Listar todas las imágenes en el directorio
image_paths = []
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.jpg') or file.endswith('.png'):  # Ajusta el formato de las imágenes
            image_paths.append(os.path.join(root, file))

print(f"Total de imágenes encontradas: {len(image_paths)}")

contrastes = []
brillos = []
image_names = []

# Procesar todas las imágenes
for image_path in image_paths:
    image = cv2.imread(image_path)

    # Verificar si la imagen fue cargada correctamente
    if image is not None:
        # Calcular contraste y brillo de la imagen
        contrast = calculate_contrast(image)
        brightness = calculate_brightness(image)

        contrastes.append(contrast)
        brillos.append(brightness)
        image_names.append(image_path)  # Guardar el nombre de la imagen
    else:
        print(f"Advertencia: No se pudo cargar la imagen {image_path}")

# Convertir listas a arrays
contrastes = np.array(contrastes)
brillos = np.array(brillos)

# Función para encontrar outliers usando IQR
def detect_outliers(data):
    Q1 = np.percentile(data, 25)  # Primer cuartil
    Q3 = np.percentile(data, 75)  # Tercer cuartil
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

# Detectar outliers para el brillo
brillo_lower_bound, brillo_upper_bound = detect_outliers(brillos)
print(f"Brillo - Limite Inferior: {brillo_lower_bound}, Limite Superior: {brillo_upper_bound}")

# Detectar outliers para el contraste
contraste_lower_bound, contraste_upper_bound = detect_outliers(contrastes)
print(f"Contraste - Limite Inferior: {contraste_lower_bound}, Limite Superior: {contraste_upper_bound}")

# Definir manualmente los límites para brillo y contraste
brillo_lower_bound = 5
brillo_upper_bound = 240

# Listas para almacenar imágenes que sean outliers
brillo_outliers = []
contraste_outliers = []
non_outliers_count = 0  # Contador para imágenes no outliers

# Iterar a través de las imágenes y detectar los outliers
for i, (brillo, contraste) in enumerate(zip(brillos, contrastes)):
    # Si la imagen es outlier en brillo o en contraste, se excluye
    if (brillo < brillo_lower_bound or brillo > brillo_upper_bound) or \
       (contraste < contraste_lower_bound or contraste > contraste_upper_bound):
        if (brillo < brillo_lower_bound or brillo > brillo_upper_bound):
            brillo_outliers.append(image_names[i])  # Guardar el nombre de la imagen con outliers de brillo
        if (contraste < contraste_lower_bound or contraste > contraste_upper_bound):
            contraste_outliers.append(image_names[i])  # Guardar el nombre de la imagen con outliers de contraste
    else:
        # Mover las imágenes que no sean outliers a la carpeta de salida
        shutil.copy(image_names[i], output_dir)
        non_outliers_count += 1  # Aumentar el contador de imágenes no outliers

# Mostrar los resultados
print(f"Imágenes con outliers de brillo: {brillo_outliers}")
print(f"Total de imágenes con outliers de brillo: {len(brillo_outliers)}")

print(f"Total de imágenes no outliers movidas: {non_outliers_count}")
